In [ ]:
import jukebox
import torch as t
from jukebox.make_models import make_vqvae, make_prior, make_model
from jukebox.hparams import setup_hparams
from jukebox.sample import sample_partial_window, upsample, save_samples
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.logger import init_logging
from jukebox.utils.torch_utils import empty_cache

# Setup device and hyperparameters
dist = setup_dist_from_mpi()
fp16 = True
hps = setup_hparams(name='5b_lyrics', levels=3, sample_length=1048576, fp16=fp16)

# Make models
vqvae, priors = make_model(hps, device=dist.device)

# Load model weights
vqvae.eval()
for prior in priors:
    prior.eval()

# Define the metadata (artist, genre, lyrics)
artist = "The Beatles"
genre = "Rock"
lyrics = "Hey Jude, don't make it bad. Take a sad song and make it better..."

# Sample the music
sample_hps = setup_hparams(name='5b_lyrics', sample_length=1048576, fp16=fp16)
zs = [t.zeros(0, dtype=t.long, device=dist.device) for _ in range(3)]
labels = sample_hps.labels[0]
sample = sample_partial_window(zs, labels, vqvae, priors, hps)

# Save the sample
save_samples(sample, sample_hps, labels, vqvae, "generated_music")
